In [3]:
import pandas as pd

# Load CSV
df = pd.read_csv('../dat/porto.csv')

# Create a summary DataFrame
summary = pd.DataFrame(columns=['Column', 'Nulls', 'Unique Values', 'Repeated Values'])


In [4]:
for col in df.columns:
    nulls = df[col].isnull().sum()
    unique_vals = df[col].nunique(dropna=True)
    repeated = len(df[col]) - unique_vals - nulls  # Total minus unique values and nulls
    summary = pd.concat([summary,
                         pd.DataFrame([[col, nulls, unique_vals, repeated]],
                                             columns=['Column', 'Nulls', 'Unique Values', 'Repeated Values'])],
                        ignore_index=True)

# Show summary
print(summary)

                                       Column Nulls Unique Values  \
0  version https://git-lfs.github.com/spec/v1     0             2   

  Repeated Values  
0               0  


Important info from the output:

    - TRIP_ID has repeated values, this must change so that it can be used as a PRIMARY KEY

    - DAY_TYPE has 1 unique value so it doesn't provide additional info, therefore it can be eliminated

    - The number of taxi drivers is 448 as TAXI_ID has that number of unique values

    - CALL_TYPE doesn't have to be modified as it matches the info given in the assignment

To solve the TRIP_ID duplicates problem we will keep the row that has the earliest datetime compared to the other duplicates

In [8]:
print(df.columns.tolist())


['version https://git-lfs.github.com/spec/v1']


In [7]:
df.sort_values(by=['TRIP_ID', 'TIMESTAMP'], ascending=[True, True], inplace=True)

df.drop_duplicates(subset='TRIP_ID', keep='first', inplace=True)

print("Number of duplicated TRIP_ID after deduplication:", df['TRIP_ID'].duplicated().sum())

KeyError: 'TRIP_ID'

Eliminate the column DAY_TYPE

In [4]:
df.drop(columns=['DAY_TYPE'], inplace=True)

CHECK DATATYPES

In [5]:
df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'], errors='coerce')

expected_dtypes = {
    'TRIP_ID': 'int64',
    'CALL_TYPE': 'object',
    'ORIGIN_CALL': 'float64',  # could be int but NaNs make it float
    'ORIGIN_STAND': 'float64', # same as above
    'TAXI_ID': 'int64',
    'TIMESTAMP': 'datetime64[ns]',
    'MISSING_DATA': 'bool',
    'POLYLINE': 'object'
}

dtype_summary = pd.DataFrame(columns=['Column', 'Expected', 'Actual', 'Match'])
for col in df.columns:
    actual = str(df[col].dtype)
    expected = expected_dtypes.get(col, 'Unknown')
    match = actual == expected
    dtype_summary = pd.concat([dtype_summary,
                               pd.DataFrame([[col, expected, actual, match]],
                                            columns=['Column','Expected','Actual','Match'])],
                              ignore_index=True)

print("Datatype check per column:\n", dtype_summary)

Datatype check per column:
          Column        Expected          Actual Match
0       TRIP_ID           int64           int64  True
1     CALL_TYPE          object          object  True
2   ORIGIN_CALL         float64         float64  True
3  ORIGIN_STAND         float64         float64  True
4       TAXI_ID           int64           int64  True
5     TIMESTAMP  datetime64[ns]  datetime64[ns]  True
6  MISSING_DATA            bool            bool  True
7      POLYLINE          object          object  True


Check if the relationships between the values from CALL_TYPE, ORIGIN_CALL and ORIGIN_STAND match the rules specified

In [6]:
# Function to check if a value is a valid numeric ID
def is_valid_id(val):
    return pd.notnull(val) and isinstance(val, (int, float))

# Rule 1: CALL_TYPE 'A', ORIGIN_CALL numeric, ORIGIN_STAND null
rule_A = df[df['CALL_TYPE']=='A']
rule_A_check = rule_A['ORIGIN_CALL'].apply(lambda x: is_valid_id(x)) & rule_A['ORIGIN_STAND'].isnull()
print("CALL_TYPE 'A' compliance with datatype:", rule_A_check.mean())

# Rule 2: CALL_TYPE 'B', ORIGIN_CALL null, ORIGIN_STAND numeric
rule_B = df[df['CALL_TYPE']=='B']
rule_B_check = rule_B['ORIGIN_STAND'].apply(lambda x: is_valid_id(x)) & rule_B['ORIGIN_CALL'].isnull()
print("CALL_TYPE 'B' compliance with datatype:", rule_B_check.mean())

# Rule 3: CALL_TYPE 'C', ORIGIN_CALL null, ORIGIN_STAND null
rule_C = df[df['CALL_TYPE']=='C']
rule_C_check = rule_C['ORIGIN_CALL'].isnull() & rule_C['ORIGIN_STAND'].isnull()
print("CALL_TYPE 'C' compliance with datatype:", rule_C_check.mean())

CALL_TYPE 'A' compliance with datatype: 1.0
CALL_TYPE 'B' compliance with datatype: 0.9861813637925322
CALL_TYPE 'C' compliance with datatype: 1.0


- 100% of the rows where CALL_TYPE='A' satisfy the rule
- 98.6% of the rows where CALL_TYPE='B' violate the rule
- 100% of the rows where CALL_TYPE='C' satisfy the rule

Filter the rows that violate the rule

In [7]:
# Filter rows where CALL_TYPE is 'B'
rule_B = df[df['CALL_TYPE']=='B']

# Keep only the rows that do NOT comply with the rule:
# ORIGIN_STAND must be numeric, ORIGIN_CALL must be NULL
violations_B = rule_B[~(rule_B['ORIGIN_STAND'].apply(is_valid_id) & rule_B['ORIGIN_CALL'].isnull())]

# Number of violating rows
print("Number of violations for CALL_TYPE 'B':", len(violations_B))

# Optionally, inspect the first few violating rows
violations_B.head()

Number of violations for CALL_TYPE 'B': 11302


,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,MISSING_DATA,POLYLINE
136,1372638560620000308,B,NaN,NaN,20000308,1970-01-01 00:00:01.372638560,False,"[[-8.58564,41.148567],[-8.585667,41.148873],[-..."
280,1372650696620000320,B,NaN,NaN,20000320,1970-01-01 00:00:01.372650696,False,"[[-8.6058,41.153517],[-8.606133,41.153472],[-8..."
433,1372656129620000307,B,NaN,NaN,20000307,1970-01-01 00:00:01.372656129,False,"[[-8.63064,41.154714],[-8.630685,41.154102],[-..."
640,1372663170620000391,B,NaN,NaN,20000391,1970-01-01 00:00:01.372663170,False,"[[-8.630712,41.154966],[-8.630775,41.154867],[..."
564,1372665032620000004,B,NaN,NaN,20000004,1970-01-01 00:00:01.372665032,False,"[[-8.591319,41.156181],[-8.591463,41.156217],[..."


There are different cases that may cause this problem:
- Both CALL_TYPE and ORIGIN_CALL are null. In this case we can change CALL_TYPE to C so that it satisfies the rule
- None of the two values are null. In this case we can change ORIGIN_CALL to null so that is satisfies the rule of CALL_TYPE='B'
- ORIGIN_STAND is the one with the null value. As this case is the one that satisfies the rule when CALL_TYPE is A, we can change it's value from B to A

Get the number of rows of each case

In [9]:
mask_B = df['CALL_TYPE'] == 'B'

mask_case1 = mask_B & df['ORIGIN_CALL'].isnull() & df['ORIGIN_STAND'].isnull()
mask_case2 = mask_B & df['ORIGIN_CALL'].notnull() & df['ORIGIN_STAND'].notnull()
mask_case3 = mask_B & df['ORIGIN_CALL'].notnull() & df['ORIGIN_STAND'].isnull()

# DataFrames with violating rows (one per case)
violations_case1 = df[mask_case1].copy()
violations_case2 = df[mask_case2].copy()
violations_case3 = df[mask_case3].copy()

# Counts
total_B = mask_B.sum()
count_case1 = mask_case1.sum()
count_case2 = mask_case2.sum()
count_case3 = mask_case3.sum()
count_non_violating_B = total_B - (count_case1 + count_case2 + count_case3)

# Print summary
print("Summary of CALL_TYPE == 'B' rows and violation cases")
print("--------------------------------------------------")
print(f"Total rows with CALL_TYPE == 'B': {total_B}")
print(f"Case 1 (ORIGIN_CALL NULL & ORIGIN_STAND NULL)  : {count_case1}  ({count_case1/total_B:.2%} of B rows)")
print(f"Case 2 (ORIGIN_CALL NOT NULL & ORIGIN_STAND NOT NULL): {count_case2}  ({count_case2/total_B:.2%} of B rows)")
print(f"Case 3 (ORIGIN_CALL NOT NULL & ORIGIN_STAND NULL): {count_case3}  ({count_case3/total_B:.2%} of B rows)")


Summary of CALL_TYPE == 'B' rows and violation cases
--------------------------------------------------
Total rows with CALL_TYPE == 'B': 817881
Case 1 (ORIGIN_CALL NULL & ORIGIN_STAND NULL)  : 11302  (1.38% of B rows)
Case 2 (ORIGIN_CALL NOT NULL & ORIGIN_STAND NOT NULL): 0  (0.00% of B rows)
Case 3 (ORIGIN_CALL NOT NULL & ORIGIN_STAND NULL): 0  (0.00% of B rows)


As every violation of the rule occurs when both ORIGIN_CALL and ORIGIN_STAND were null we only change the rows as it was explained earlier

In [11]:
# Dictionary to keep logs of changes
change_log = {'Case': [], 'Index': [], 'Original_CALL_TYPE': [], 'Original_ORIGIN_CALL': [], 'Original_ORIGIN_STAND': [], 'New_CALL_TYPE': [], 'New_ORIGIN_CALL': []}

# -----------------------------
# Case 1: Both ORIGIN_CALL and ORIGIN_STAND are NULL for CALL_TYPE 'B'
mask_case1 = (df['CALL_TYPE'] == 'B') & df['ORIGIN_CALL'].isnull() & df['ORIGIN_STAND'].isnull()
for idx in df[mask_case1].index:
    change_log['Case'].append('Case 1 (B→C)')
    change_log['Index'].append(idx)
    change_log['Original_CALL_TYPE'].append(df.at[idx, 'CALL_TYPE'])
    change_log['Original_ORIGIN_CALL'].append(df.at[idx, 'ORIGIN_CALL'])
    change_log['Original_ORIGIN_STAND'].append(df.at[idx, 'ORIGIN_STAND'])
    change_log['New_CALL_TYPE'].append('C')
    change_log['New_ORIGIN_CALL'].append(df.at[idx, 'ORIGIN_CALL'])
    df.at[idx, 'CALL_TYPE'] = 'C'

Copy the cleaned dataframe

In [ ]:
# Save cleaned DataFrame to CSV
for col in df.columns:
    nulls = df[col].isnull().sum()
    unique_vals = df[col].nunique(dropna=True)
    repeated = len(df[col]) - unique_vals - nulls  # Total minus unique values and nulls
    summary = pd.concat([summary,
                         pd.DataFrame([[col, nulls, unique_vals, repeated]],
                                             columns=['Column', 'Nulls', 'Unique Values', 'Repeated Values'])],
                        ignore_index=True)

# Show summary
print(summary)

mask_B = df['CALL_TYPE'] == 'B'

mask_case1 = mask_B & df['ORIGIN_CALL'].isnull() & df['ORIGIN_STAND'].isnull()
mask_case2 = mask_B & df['ORIGIN_CALL'].notnull() & df['ORIGIN_STAND'].notnull()
mask_case3 = mask_B & df['ORIGIN_CALL'].notnull() & df['ORIGIN_STAND'].isnull()

# DataFrames with violating rows (one per case)
violations_case1 = df[mask_case1].copy()
violations_case2 = df[mask_case2].copy()
violations_case3 = df[mask_case3].copy()

# Counts
total_B = mask_B.sum()
count_case1 = mask_case1.sum()
count_case2 = mask_case2.sum()
count_case3 = mask_case3.sum()
count_non_violating_B = total_B - (count_case1 + count_case2 + count_case3)

# Print summary
print("Summary of CALL_TYPE == 'B' rows and violation cases")
print("--------------------------------------------------")
print(f"Total rows with CALL_TYPE == 'B': {total_B}")
print(f"Case 1 (ORIGIN_CALL NULL & ORIGIN_STAND NULL)  : {count_case1}  ({count_case1/total_B:.2%} of B rows)")
print(f"Case 2 (ORIGIN_CALL NOT NULL & ORIGIN_STAND NOT NULL): {count_case2}  ({count_case2/total_B:.2%} of B rows)")
print(f"Case 3 (ORIGIN_CALL NOT NULL & ORIGIN_STAND NULL): {count_case3}  ({count_case3/total_B:.2%} of B rows)")

Change datatype of ORIGIN_CALL and ORIGIN_STAND from float to int

In [ ]:
df["ORIGIN_CALL"] = df["ORIGIN_CALL"].astype("Int64")
df["ORIGIN_STAND"] = df["ORIGIN_STAND"].astype("Int64")

Check the max number of decimal in the dataset

In [ ]:
import ast

def count_decimals(num):
    """Return the number of decimals in a float, ignoring scientific notation."""
    s = str(num)
    if '.' in s:
        return len(s.split('.')[-1].rstrip('0'))  # remove trailing zeros
    return 0

def max_decimals_in_polyline(polyline_str):
    """Given a POLYLINE string (list of [lon, lat]), return max decimals in any point."""
    if not polyline_str or polyline_str == '[]':
        return 0
    try:
        coords = ast.literal_eval(polyline_str)
        return max(
            count_decimals(coord)
            for point in coords
            for coord in point
        )
    except Exception:
        return 0

# Apply function to all rows and find global max
max_decimals = df['POLYLINE'].apply(max_decimals_in_polyline).max()

print(f"Maximum number of decimal places in POLYLINE: {max_decimals}")

df.to_csv("../dat/clean.csv", index=False)